# Analysis of the Dataset Extension Attack

In [12]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import re

In [13]:
# Load data
df = pd.read_csv("formatted_results.csv")

# Extract numeric dataset size (optional)
def extract_dataset_size(name):
    match = re.search(r"(\d+)k", name)
    if match:
        return int(match.group(1)) * 1000
    elif "euro" in name:
        return 25000  # placeholder
    return None

df["DatasetSize"] = df["Dataset"].apply(extract_dataset_size)
df["Overlap"] = df["Overlap"].astype(float)

# Output directory
os.makedirs("analysis/plots", exist_ok=True)
os.makedirs("analysis/tables", exist_ok=True)

# Metrics to visualize
metrics = [
    ("TrainedF1", "F1 Score"),
    ("ReidentificationRate", "Re-identification Rate"),
    ("TrainedRecall", "Recall"),
    ("TrainedPrecision", "Precision"),
    ("TotalRuntime", "Total Runtime (min)")
]

# Baseline metrics per dataset (dict format)
baseline_metrics = {
    "fakename_1k":     {"Precision": 0.2162, "Recall": 0.2476, "F1": 0.2300},
    "fakename_2k":     {"Precision": 0.2131, "Recall": 0.2452, "F1": 0.2271},
    "fakename_5k":     {"Precision": 0.2144, "Recall": 0.2470, "F1": 0.2287},
    "fakename_10k":    {"Precision": 0.2151, "Recall": 0.2467, "F1": 0.2289},
    "fakename_20k":    {"Precision": 0.2153, "Recall": 0.2473, "F1": 0.2293},
    "fakename_50k":    {"Precision": 0.2151, "Recall": 0.2463, "F1": 0.2288},
    "euro_person":     {"Precision": 0.2197, "Recall": 0.2446, "F1": 0.2306}
}

In [17]:
# Loop through each (Encoding, Dataset) pair
for (encoding, dataset), group in df.groupby(["Encoding", "Dataset"]):
    fig, axes = plt.subplots(nrows=2, ncols=4, figsize=(20, 8))

    dataset_label = dataset.replace(".tsv", "").replace("_", " ")
    title = f"{encoding} — {dataset_label}"
    fig.suptitle(title, fontsize=16)

    dataset_key = dataset.replace(".tsv", "")

    # Regular metric plots
    for ax, (metric_key, metric_label) in zip(axes.flat[:6], metrics):
        sns.lineplot(
            data=group,
            x="Overlap",
            y=metric_key,
            hue="DropFrom",
            marker="o",
            ax=ax
        )
        ax.set_title(metric_label)
        ax.set_xlabel("Overlap")
        ax.grid(True)

        # Baselines
        if dataset_key in baseline_metrics:
            if metric_key == "TrainedF1":
                ax.axhline(y=baseline_metrics[dataset_key]["F1"], linestyle="--", color="gray", label="Baseline F1")
                ax.legend()
                ax.set_title("F1 & Dice Score")
            elif metric_key == "TrainedRecall":
                ax.axhline(y=baseline_metrics[dataset_key]["Recall"], linestyle="--", color="gray", label="Baseline Recall")
                ax.legend()
            elif metric_key == "TrainedPrecision":
                ax.axhline(y=baseline_metrics[dataset_key]["Precision"], linestyle="--", color="gray", label="Baseline Precision")
                ax.legend()

    # ➕ Updated: Two subplots for Re-ID Comparison by DropFrom
    melted = group.melt(
        id_vars=["Overlap", "DropFrom"],
        value_vars=["ReidentificationRateFuzzy", "ReidentificationRateGreedy", "ReidentificationRate"],
        var_name="Method",
        value_name="Rate"
    )
    method_map = {
        "ReidentificationRateFuzzy": "Fuzzy",
        "ReidentificationRateGreedy": "Greedy",
        "ReidentificationRate": "Combined"
    }
    melted["Method"] = melted["Method"].map(method_map)
    melted["Rate"] *= 100

    # Plot for DropFrom = Eve
    ax = axes.flat[5]
    subset_eve = melted[melted["DropFrom"] == "Eve"]
    sns.lineplot(
        data=subset_eve,
        x="Overlap",
        y="Rate",
        hue="Method",
        marker="o",
        ax=ax
    )
    ax.set_title("Re-ID (DropFrom = Eve)")
    ax.set_ylabel("Re-ID Rate (%)")
    ax.grid(True)

    # Plot for DropFrom = Both
    ax = axes.flat[6]
    subset_both = melted[melted["DropFrom"] == "Both"]
    sns.lineplot(
        data=subset_both,
        x="Overlap",
        y="Rate",
        hue="Method",
        marker="o",
        ax=ax
    )
    ax.set_title("Re-ID (DropFrom = Both)")
    ax.set_ylabel("Re-ID Rate (%)")
    ax.grid(True)

    # ➕ Subplot: Trained F1 vs. HypOp F1 (no DropFrom/Overlap)
    ax = axes.flat[7]
    sns.scatterplot(
        data=group,
        x="HypOpF1",
        y="TrainedF1",
        s=60,
        color="steelblue",
        ax=ax
    )
    ax.plot([0, 1], [0, 1], linestyle="--", color="gray", label="x = y")
    ax.set_title("Trained vs. HypOp F1")
    ax.set_xlabel("HypOp F1 (Validation)")
    ax.set_ylabel("Trained F1 (Full Data)")
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.grid(True)
    ax.legend(loc="lower right", fontsize="small")


    plt.tight_layout(rect=[0, 0, 1, 0.95])
    filename = f"analysis/plots/{encoding}_{dataset_label.replace(' ', '_')}_metrics.png"
    plt.savefig(filename, dpi=300)
    plt.close()


In [15]:
# Select relevant columns for the summary
summary_df = df[[
    "Dataset", "Encoding", "Overlap", "DropFrom",
    "ReidentificationRateFuzzy", "ReidentificationRateGreedy", "ReidentificationRate"
]]

# Rename columns for clarity
summary_df = summary_df.rename(columns={
    "ReidentificationRateFuzzy": "Fuzzy Rate",
    "ReidentificationRateGreedy": "Greedy Rate",
    "ReidentificationRate": "Combined Rate"
})

# Sort for readability
summary_df = summary_df.sort_values(by=["Dataset", "Encoding", "Overlap", "DropFrom"])

# Export to CSV
summary_df.to_csv("analysis/tables/reidentification_summary.csv", index=False)

# Display as a table
summary_df.reset_index(drop=True, inplace=True)


